In [1]:
%load_ext autoreload

In [2]:
import numpy as np
import os
import mcubes
import meshcat
import pydrake
from pydrake.geometry import SceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser, LoadModelDirectives, ProcessModelDirectives
from pydrake.multibody.tree import RevoluteJoint
from pydrake.all import ConnectMeshcatVisualizer, InverseKinematics, RigidTransform, RotationMatrix
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra
import time
from meshcat import Visualizer
from functools import partial

import ipywidgets as widgets
from IPython.display import display

from pydrake.all import MeshcatVisualizerCpp, MeshcatVisualizerParams, Role
import rrt, prm, utils, rrtiris
from sandbox.t_space_utils import (convert_t_to_q, 
                                   convert_q_to_t)

from utils import meshcat_line
from iris_t_space import set_up_iris_t_space

In [3]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])

# Build plant and simulation

In [4]:
# Build plant and simulation#settings
q0 = [0.0, 0.0, 0.0]
q_low = [-1.7,-1.7, -2.0]
q_high = [1.7, 1.7, 2.0]
t_low = convert_q_to_t(np.array(q_low).reshape(1,-1)).squeeze()
t_high = convert_q_to_t(np.array(q_high).reshape(1,-1)).squeeze()

#marching cubes
q_low_mc = q_low.copy()
q_high_mc =  q_high.copy()
N = 50

In [5]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
tworob_asset = FindResourceOrThrow("drake/sandbox/assets/doublerob.urdf")
#two_dof_asset = FindResourceOrThrow("drake/sandbox/assets/planar2dof.urdf")
box_asset = FindResourceOrThrow("drake/sandbox/assets/box.urdf")

models =[]
models.append(parser.AddModelFromFile(tworob_asset))
#models.append(parser.AddModelFromFile(one_dof_asset))
models.append(parser.AddModelFromFile(box_asset))

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            idx += 1
            
locs = [[0.,0.,0.],[0.,0.,0.]]
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform(locs[idx]))
    idx+=1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False, )

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

joints = []
idx = 0
for model in models:
    jointindx = plant.GetJointIndices(model)
    for j in jointindx:
        joint = plant.get_mutable_joint(j)
        if isinstance(joint, RevoluteJoint):
            joints.append(joint)
            joints[-1].set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([np.array([q_high[idx]])]))
            idx +=1
        
    
def set_joint_ang(val, idx):
    joints[idx].set_angle(plant_context, val)
    
def set_joint_angles(vals):
    joints[0].set_angle(plant_context, vals[0])
    joints[1].set_angle(plant_context, vals[1])
    joints[2].set_angle(plant_context, vals[2])
    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)

def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))

col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

def eval_cons_rational(t0, t1, t2, c, tol):
    q = convert_t_to_q(np.array([t0, t1, t2]).reshape(1,-1)).squeeze() 
    return col_func_handle(*q)
   
col_func_handle_rational = partial(eval_cons_rational, c=collision_constraint, tol=0.01)

def showres(q):
    set_joint_ang(q[0],0)
    set_joint_ang(q[1],1)
    set_joint_ang(q[2],2)
    col = col_func_handle(*q)
    t = convert_q_to_t(np.array(q).reshape(1,-1)).squeeze()
    if col:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(t))
    else:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0x3EFF00))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(t))
    diagram.Publish(context)
    print("              ", end = "\r")
    print(col , end = "\r")

sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7011/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6010...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/
Connected to meshcat-server.


In [6]:
#marching cubes
N = 50
vertices, triangles = mcubes.marching_cubes_func(tuple(t_low), tuple(t_high), N, N, N, col_func_handle_rational, 0.5)
vis2["collision_constraint"].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            meshcat.geometry.MeshLambertMaterial(color=0xff0000, wireframe=True))
q = q0.copy()
showres(q)

In [7]:
for slider in sliders:
    display(slider)

display(vis.jupyter_cell())
display(vis2.jupyter_cell())

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q2', max=2.0, min=-2.0)

In [14]:
#SPP + IRIS / RRT / PRM  CONFIG + viz
start_q = np.array([0.2, -1.6, 1.5])
target_q = np.array([0.8,-0.9,1.5])
start = convert_q_to_t(start_q.reshape(1,-1))
target = convert_q_to_t(target_q.reshape(1,-1))

#
#plot start and target
mat = meshcat.geometry.MeshLambertMaterial(color=0xFFDD36)
mat.reflectivity = 1.0
vis2['start'].set_object(
                meshcat.geometry.Sphere(0.03), mat)
vis2['start'].set_transform(
                meshcat.transformations.translation_matrix(start.reshape(-1,)))

mat = meshcat.geometry.MeshLambertMaterial(color=0x06D300)
mat.reflectivity = 1.0
vis2['target'].set_object(
                meshcat.geometry.Sphere(0.03), mat)
vis2['target'].set_transform(
                meshcat.transformations.translation_matrix(target.reshape(-1,)))

def draw_traj_tspace(traj, maxit, name):
    #evals end twice fix later
    for it in range(maxit):
        pt = traj.value(it*traj.end_time()/maxit)
        pt_nxt = traj.value((it+1)*traj.end_time()/maxit)
        
        pt_q = convert_t_to_q(pt.reshape(1,-1)).squeeze()

        mat = meshcat.geometry.MeshLambertMaterial(color=0xFFF812)
        mat.reflectivity = 1.0
        vis2[name]['traj']['points' + str(it)].set_object( meshcat_line(pt.squeeze(), pt_nxt.squeeze(),width = 0.03), mat)
        
        set_joint_angles(pt_q.reshape(-1,))
        tf_l2 = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(3)))
        R_l2 = tf_l2.rotation()
        tl_l2 = R_l2@np.array([0,0,0.9]) + tf_l2.translation()

        tf_la = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(4)))
        R_la = tf_la.rotation()
        tl_la = R_la@np.array([0,0,1.2]) + tf_la.translation()


        mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
        mat.reflectivity = 1.0
        vis[name]['traj']['link2']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.02), mat)
        vis[name]['traj']['link2']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(tl_l2))
        mat = meshcat.geometry.MeshLambertMaterial(color=0x07F100)
        mat.reflectivity = 1.0
        vis[name]['traj']['linka']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.02), mat)
        vis[name]['traj']['linka']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(tl_la))

## CONSTRUCT T SPACE IRIS ##


In [61]:
iris_rational_space, query, forward_kin = set_up_iris_t_space2(plant, scene_graph, context, settings = None)

def do_iris(t_seed, verbose = True):
        start_time = time.time()
        hpoly = iris_rational_space(query, t_seed, require_containment_points=[t_seed], iteration_limit=100)
        ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
        if verbose:
            print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
              ellipse.center(), flush=True)
        return hpoly, ellipse

In [72]:

seed_points_q = np.array([[0.0, 0, 0], # startpoint
                        [0.8, -0.8, 1.3],  # blue low green up
                        [0.1, -1.2, 0.9],     # green low other up
                        [0.2, -0.6, 1.6],
                        [-0.5, -1.0, 1.9]])[::-1, :]    # passing

seed_points = convert_q_to_t(seed_points_q)

seed = np.array([0.3, -1, -0.1])
do_iris(seed.squeeze())

0
1
Time:   2.02 	Volume:   0.17 	Center: [ 0.36813047 -0.76115427 -0.23206563]


(<pydrake.geometry.optimization.HPolyhedron at 0x7f8a437049f0>,
 <pydrake.geometry.optimization.Hyperellipsoid at 0x7f8a437487f0>)

In [9]:
USE_HAND_CRAFTED = True

seed_points_q = np.array([[0.0, 0, 0], # startpoint
                        [0.8, -0.8, 1.3],  # blue low green up
                        [0.1, -1.2, 0.9],     # green low other up
                        [0.2, -0.6, 1.6],
                        [-0.5, -1.0, 1.9]])[::-1, :]    # passing

seed_points = convert_q_to_t(seed_points_q)

def rejection_sampling_iris(its):
    regions = []
    ellipses = []
    samples = []
    seed_points = [start.squeeze(), target.squeeze()]
    reg, ell = do_iris(seed_points[0])
    regions.append(reg)
    ellipses.append(ell)
    reg, ell = do_iris(seed_points[1])
    regions.append(reg)
    ellipses.append(ell)
    
    for _ in range(its):
        #rejection sampling to get initial feasible point 
        found = False
        while not found:
            t = np.random.rand(3)
            t_samp = (1-t)*t_low + t*t_high
            found = (col_func_handle_rational(*t_samp)==0.0)
        print("point found: ", t_samp)
        seed_points.append(t_samp)
        reg, ell = do_iris(t_samp)
    
        regions.append(reg)
        ellipses.append(ell)
    return regions, ellipses, seed_points

def hand_crafted_seedpoint_iris(seed_points):
    regions = []
    ellipses = []
    for i in range(seed_points.shape[0]):
        reg, ell = do_iris(seed_points[i, :])
        regions.append(reg)
        ellipses.append(ell)
    print("SUCCESS")
    return regions, ellipses, seed_points 

if USE_HAND_CRAFTED:
    regions, ellipses, region_seeds = hand_crafted_seedpoint_iris(seed_points)
else:
    regions, ellipses, region_seeds = rejection_sampling_iris(15)

region_seeds = np.array(region_seeds)
for i in range(region_seeds.shape[0]):
    vis2['iris']['seedpoints']["seedpoint"+str(i)].set_object(
                meshcat.geometry.Sphere(0.05), meshcat.geometry.MeshLambertMaterial(color=0x0FB900))
    vis2['iris']['seedpoints']["seedpoint"+str(i)].set_transform(
                meshcat.transformations.translation_matrix(region_seeds[i,:]))

0
terminating because a required containment point would have not been contained
0
1
0
1
0
1
0
1
SUCCESS


In [10]:
iris_hand = partial(do_iris, verbose = False)

def plot_callback(region, seed_point, pos_samp, region_id, vis):
    scale = np.clip(150 + region_id *105/5.0, a_min = 0, a_max = 255)
    mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((int(0.2*scale), 155, int(scale))) , wireframe=True)
    mat_seed = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((0, 0, 0)), wireframe=True)
    mat_samp = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((155, 0, 155)), wireframe=True)
    mat.opacity = 0.2
    
    utils.plot_3d_poly(region = region,
                       resolution = 30,
                       vis = vis['regions'],
                       name = str(region_id),
                       mat = mat)
    
    utils.plot_point(seed_point,
                     radius = 0.05, 
                     mat = mat_seed, 
                     vis = vis['seed'], 
                     marker_id = str(region_id))
    
    utils.plot_point(pos_samp,
                     radius = 0.05, 
                     mat = mat_samp, 
                     vis = vis['samples'], 
                     marker_id = str(region_id))
    
plot_callback_handle = partial(plot_callback, vis = vis2['rrtiris']) 

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1], pos[2])
    
rrti_col_fn_handle = partial(collision, col_func_handle = col_func_handle_rational)

RRTI = rrtiris.RRTIRIS(start = start.squeeze(), 
                       goal = target.squeeze(), 
                       limits = [np.array(t_low), np.array(t_high)], 
                       iris_handle = iris_hand,
                       col_func_handle = rrti_col_fn_handle, 
                       init_goal_sample_rate = 0.05,
                       goal_sample_rate_scaler = 0.8,
                       verbose = True,
                       plotcallback = plot_callback_handle,
                       sample_collision_free = True
                       )
success, regions, ellipses = RRTI.run(20)

0
1
0
0
1
0.37929966940357
[RRT IRIS] it: 0 distance to target:  0.379 goalsample prob:  0.497
1
0
1
0.3792996694035699
[RRT IRIS] it: 1 distance to target:  0.379 goalsample prob:  0.497
2
0
terminating because a required containment point would have not been contained
0.37929966940357007
3
0
1
0.332118461424663
[RRT IRIS] it: 3 distance to target:  0.332 goalsample prob:  0.534
4
0
terminating because a required containment point would have not been contained
0.3113164453446489
[RRT IRIS] it: 4 distance to target:  0.311 goalsample prob:  0.551
5
0
1
0.3113164453446489
6
0
terminating because a required containment point would have not been contained
0.2730206310722911
[RRT IRIS] it: 6 distance to target:  0.273 goalsample prob:  0.582
7
0
1
0.2730206310722911
8
0
terminating because a required containment point would have not been contained
1.2412670766236366e-16
[RRT IRIS] it: 8 distance to target:  0.000 goalsample prob:  0.800


## Plotting Resulting Regions ##


In [27]:
idx = 0

for region in regions:
    c1 = int(np.clip(255*np.random.rand(), a_min = 0, a_max = 255))
    c2 = int(np.clip(255*np.random.rand(), a_min = 0, a_max = 255))
    c3 = int(np.clip(255*np.random.rand(), a_min = 0, a_max = 255))
    mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((c1, c2,c3)), wireframe=True)
    mat.opacity = 0.3
    utils.plot_3d_poly(region = region,
                       resolution = 30,
                       vis = vis2['iris']['regions'],
                       name = str(idx),
                       mat = mat)
    
    C = ellipse.A()#[:, (0,2,1)]
    d = ellipse.center()#[[0,2,1]]
    radii, R = np.linalg.eig(C.T@C)
    R[:,0] = R[:,0]*np.linalg.det(R)
    Rot = RotationMatrix(R)
 
    transf = RigidTransform(Rot, d)
    mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((c1, c2,c3)), wireframe=True)
    mat.opacity = 0.15
    vis2['iris']['ellipses'][str(idx)].set_object(
            meshcat.geometry.Ellipsoid(np.divide(1,np.sqrt(radii))),
            mat)

    vis2['iris']['ellipses'][str(idx)].set_transform(transf.GetAsMatrix4())
    
    idx+=1

NameError: name 'regions' is not defined

In [38]:
def plot_region(region, ellipse, vis, name):
    c1 = 0
    c2 = 0
    c3 = 200
    mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((c1, c2,c3)), wireframe=True)
    mat.opacity = 0.3
    utils.plot_3d_poly(region = region,
                       resolution = 30,
                       vis = vis2['iris']['regions'],
                       name = name,
                       mat = mat)
    
    C = ellipse.A()#[:, (0,2,1)]
    d = ellipse.center()#[[0,2,1]]
    radii, R = np.linalg.eig(C.T@C)
    R[:,0] = R[:,0]*np.linalg.det(R)
    Rot = RotationMatrix(R)
 
    transf = RigidTransform(Rot, d)
    c1 = 0
    c2 = 255
    c3 = 200
    mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((c1, c2,c3)), wireframe=True)
    mat.opacity = 0.8
    vis2['iris']['ellipses'][name].set_object(
            meshcat.geometry.Ellipsoid(np.divide(1,np.sqrt(radii))),
            mat)

    vis2['iris']['ellipses'][name].set_transform(transf.GetAsMatrix4())
    

In [39]:
# Solve path planning
start_time = time.time()
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(start.squeeze(), target.squeeze(), regions)
spp.set_max_velocity([.3, .3, .3])
spp.set_extra_control_points_per_region(3)

# print(spp.num_regions())
traj_iris = spp.SolveVerbose()
print(time.time() - start_time)
print(traj_iris.start_time())
print(traj_iris.end_time())
draw_traj_tspace(traj_iris, 100, 'iris')

NameError: name 'regions' is not defined

In [40]:
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere, VPolytope,
    Iris, IrisOptions, MakeIrisObstacles, Variable,
    BsplineTrajectoryThroughUnionOfHPolyhedra,
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff, InverseKinematics,
    RationalForwardKinematics, FindBodyInTheMiddleOfChain
)

In [60]:
def set_up_iris_t_space2(plant, scene_graph, context, settings = None):
    

    #hardcoded
    dReal_polytope_tol = 1e-10
    starting_vol_eps = 1e-3


    forward_kin = RationalForwardKinematics(plant)
    query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))
    q_star = np.zeros(forward_kin.t().shape[0])

    def convert_RationalForwardPoseList_to_TransformExpressionList(pose_list):
        ret = []
        for p in pose_list:
            ret.append(p.asRigidTransformExpr())
        return ret

    def MakeFromSceneGraph(query, geom, expressed_in=None):
        shape = query.inspector().GetShape(geom)
        if isinstance(shape, (Sphere, Ellipsoid)):
            return Hyperellipsoid(query, geom, expressed_in)
        return HPolyhedron(query, geom, expressed_in)

    def GrowthVolumeRational(E, X_WA, X_WB, setA, setB, A, b, guess=None):
        snopt = SnoptSolver()
        prog = MathematicalProgram()
        t = forward_kin.t()
        prog.AddDecisionVariables(t)
        
        if guess is not None:
            prog.SetInitialGuess(t, guess)

        prog.AddLinearConstraint(A, b-np.inf, b, t)
        p_AA = prog.NewContinuousVariables(3, "p_AA")
        p_BB = prog.NewContinuousVariables(3, "p_BB")
        setA.AddPointInSetConstraints(prog, p_AA)
        setB.AddPointInSetConstraints(prog, p_BB)
        prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), t)

        p_WA = X_WA.multiply(p_AA+0)

        p_WB = X_WB.multiply(p_BB+0)

        prog.AddConstraint(eq(p_WA, p_WB))
        result = snopt.Solve(prog)
        return result.is_success(), result.get_optimal_cost(), result.GetSolution(t)

    def TangentPlane(self, point):
        a = 2 * self.A().T @ self.A() @ (point - self.center())
        a = a / np.linalg.norm(a)
        b = a.dot(point)
        return a, b

    Hyperellipsoid.TangentPlane = TangentPlane

    def iris_rational_space(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
        dim = plant.num_positions()
        lb = plant.GetPositionLowerLimits()
        rational_lb = forward_kin.ComputeTValue(lb, q_star)
        ub = plant.GetPositionUpperLimits()
        rational_ub = forward_kin.ComputeTValue(ub, q_star)
        volume_of_unit_sphere = 4.0*np.pi/3.0
        
        E = Hyperellipsoid(np.eye(3)/starting_vol_eps, point)

        best_volume = starting_vol_eps**dim * volume_of_unit_sphere
        

        max_faces = 25
        
        link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                            plant.world_body().index())
        X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
        X_WB_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

        inspector = query.inspector()
        pairs = inspector.GetCollisionCandidates()

        P = HPolyhedron.MakeBox(rational_lb, rational_ub)
        A = np.vstack((P.A(), np.zeros((max_faces*len(pairs),3))))  # allow up to 10 faces per pair.
        b = np.concatenate((P.b(), np.zeros(max_faces*len(pairs))))

        geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
        sets = {geom:MakeFromSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
        body_indexes_by_geom_id = {geom:
                                plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 
        
        #Turn onto true to certify regions using SOS at each iteration.
        certify = False
        # refine polytopes if not certified collision free
        refine = False and certify
            
        iteration = 0
        num_faces = 2*len(lb)
        while True:
            ## Find separating hyperplanes

            for geomA, geomB in pairs:
                #print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
                # Run snopt at the beginning
                while True:
                    X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
                    X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
                    success, growth, qstar = GrowthVolumeRational(E,
                        X_WA, X_WB,
                        sets[geomA], sets[geomB], 
                        A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, 
                        point)
                    if success:
                        #print(f"snopt_example={qstar}, growth = {growth}")
                        # Add a face to the polytope
                        A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                        num_faces += 1
                        if num_faces > max_faces:
                            break
                        #     A = np.vstack((A, np.zeros((max_faces*len(pairs),3))))  # allow up to 10 faces per pair.
                        #     b = np.concatenate((b, np.zeros(max_faces*len(pairs))))
                    else:
                        break

                if certify:
                    pass
                
        
            if any([np.any(A[:num_faces,:] @ p > b[:num_faces]) for p in require_containment_points]):
                print("terminating because a required containment point would have not been contained")
                break

            P = HPolyhedron(A[:num_faces,:],b[:num_faces])

            E = P.MaximumVolumeInscribedEllipsoid()
            print(iteration)
            plot_region(P, E, vis2, 'iterplot')
            iteration += 1
            if iteration >= iteration_limit:
                break

            volume = volume_of_unit_sphere / np.linalg.det(E.A())
            if volume - best_volume <= termination_threshold:
                break
            best_volume = volume

        return P
    return iris_rational_space, query, forward_kin
